In [156]:
import pandas as pd 
import re
import pandas as pd
import json
import markdown
import io
#from weasyprint import HTML
#from converter import md2df, df2json

import os
home = os.path.expanduser("~")

print('OS path is set to:', home)

OS path is set to: /Users/sofia


In [137]:
df = pd.read_parquet('cm_features_to_oct2019.parquet')

In [138]:
#get all the column names from your data
list(df.columns)

['gleditsch_ward',
 'ged_sb',
 'ged_ns',
 'ged_os',
 'acled_sb',
 'acled_sb_count',
 'acled_os',
 'ged_sb_tsum_24',
 'wdi_sp_pop_totl',
 'ged_sb_tlag_1',
 'ged_sb_tlag_2',
 'ged_sb_tlag_3',
 'ged_sb_tlag_4',
 'ged_sb_tlag_5',
 'ged_sb_tlag_6',
 'ged_os_tlag_1',
 'wdi_ag_lnd_frst_k2',
 'wdi_dt_oda_odat_pc_zs',
 'wdi_ms_mil_xpnd_gd_zs',
 'wdi_ms_mil_xpnd_zs',
 'wdi_nv_agr_totl_kd',
 'wdi_nv_agr_totl_kn',
 'wdi_ny_gdp_pcap_kd',
 'wdi_sp_dyn_le00_in',
 'wdi_se_enr_prim_fm_zs',
 'wdi_se_enr_prsc_fm_zs',
 'wdi_se_prm_nenr',
 'wdi_sh_sta_maln_zs',
 'wdi_sh_sta_stnt_zs',
 'wdi_sl_tlf_totl_fe_zs',
 'wdi_sm_pop_refg_or',
 'wdi_sm_pop_netm',
 'wdi_sm_pop_totl_zs',
 'wdi_sp_dyn_imrt_in',
 'wdi_sh_dyn_mort_fe',
 'wdi_sp_pop_14_fe_zs',
 'wdi_sp_pop_1564_fe_zs',
 'wdi_sp_pop_65up_fe_zs',
 'wdi_sp_pop_grow',
 'wdi_sp_urb_totl_in_zs',
 'splag_wdi_sl_tlf_totl_fe_zs',
 'splag_wdi_sm_pop_refg_or',
 'splag_wdi_sm_pop_netm',
 'splag_wdi_ag_lnd_frst_k2',
 'vdem_v2x_delibdem',
 'vdem_v2x_egaldem',
 'vdem_v2x_

In [139]:
#create empty df
df_new = pd.DataFrame()
#get all variable names as rows
df_new['variable_name']= df.columns.values.tolist() 
df_new

,variable_name
0,gleditsch_ward
1,ged_sb
2,ged_ns
3,ged_os
4,acled_sb
...,...
121,renewable_internal_pcap_t48
122,renewable_pcap_t48
123,splag_1_decay_ged_sb_5
124,splag_1_decay_ged_os_5


In [140]:
#match the variable names with the descriptions
## you can change it to match the transfroms, so "if 'splag' in df_name: header = 'spatial lag'"
## you can repeat it for as many columns you need
#header= ''
#def give_me_transforms(df_new):
 #   global header
 #   if "splag" in df_new: header = 'spatial lag'
 #  if "decay" in df_new: header = 'decay function'
 #   if "t48" in df_new: header = 'temporal lag of 48 months'
 #   if "tlag" in df_new: header = 'temporal lag of X months'
 #   if "tsum_24" in df_new: header = '24-month moving sum' 
 #   return header

In [141]:
def give_me_transforms(df_new):
    headers = []
    if "splag" in df_new: headers.append('spatial lag')
    if "decay" in df_new: headers.append('decay function')
    if "t48" in df_new: headers.append('temporal lag of 48 months')
    if "tlag" in df_new: headers.append('temporal lag of X months')
    if "tsum_24" in df_new: headers.append('24-month moving sum')
    return headers

In [142]:
# Generate the transform_description column based on the variable_name column
df_new['transform_description'] = df_new['variable_name'].apply(give_me_transforms)

# Combine the list of header strings for each column into a single string
df_new['transform_description'] = df_new['transform_description'].apply(lambda x: ", ".join(x))

# Print the updated DataFrame
print(df_new)

                   variable_name        transform_description
0                 gleditsch_ward                             
1                         ged_sb                             
2                         ged_ns                             
3                         ged_os                             
4                       acled_sb                             
..                           ...                          ...
121  renewable_internal_pcap_t48    temporal lag of 48 months
122           renewable_pcap_t48    temporal lag of 48 months
123       splag_1_decay_ged_sb_5  spatial lag, decay function
124       splag_1_decay_ged_os_5  spatial lag, decay function
125       splag_1_decay_ged_ns_5  spatial lag, decay function

[126 rows x 2 columns]


In [143]:
#df_new['transform_description'] = df_new['variable_name'].apply(give_me_transforms)
#print(df_new)

In [144]:
#match the variable names with the descriptions
## you can change it to match the descriptions, so "if 'wdi' in df_name: header = 'world development index'"
## you can repeat it for as many columns you need 
header= ''
def give_me_description(df_new):
    global header
    if "acled" in df_new: header = 'Armed Conflict Location Event Dataset'
    if "wdi" in df_new: header = "World Development Index"
    if "vdem" in df_new: header = "Varieties of Democracy (vdem)"
    if "agr_withdrawal" in df_new: header = "AQUASTAT Glossary, FAO, 2019"  
    if "dam_cap_pcap" in df_new: header = "AQUASTAT Glossary, FAO, 2019" 
    if "groundwater_export" in df_new: header = "AQUASTAT Glossary, FAO, 2019"
    if "fresh_withdrawal" in df_new: header = "AQUASTAT Glossary, FAO, 2019"
    if "ind_efficiency" in df_new: header = "FAO. [2022]. AQUASTAT Core Database" 
    if "irr_agr_efficiency" in df_new: header = "FAO. [2022]. AQUASTAT Core Database" 
    if "services_efficiency" in df_new: header = "FAO. [2022]. AQUASTAT Core Database" 
    if "general_efficiency" in df_new: header = "FAO. [2022]. AQUASTAT Core Database"
    if "water_stress" in df_new: header = "FAO. [2022]. AQUASTAT Core Database"
    if "renewable_internal_pcap" in df_new: header = "FAO. [2022]. AQUASTAT Core Database"
    if "renewable_pcap" in df_new: header = "FAO. [2022]. AQUASTAT Core Database"
    if "ged" in df_new: header = "UCDP Georeferenced Events Dataset (UCDP GED)"
    return header

In [145]:
df_new['source_description'] = df_new['variable_name'].apply(give_me_description)
print(df_new)

                   variable_name        transform_description  \
0                 gleditsch_ward                                
1                         ged_sb                                
2                         ged_ns                                
3                         ged_os                                
4                       acled_sb                                
..                           ...                          ...   
121  renewable_internal_pcap_t48    temporal lag of 48 months   
122           renewable_pcap_t48    temporal lag of 48 months   
123       splag_1_decay_ged_sb_5  spatial lag, decay function   
124       splag_1_decay_ged_os_5  spatial lag, decay function   
125       splag_1_decay_ged_ns_5  spatial lag, decay function   

                               source_description  
0                                                  
1    UCDP Georeferenced Events Dataset (UCDP GED)  
2    UCDP Georeferenced Events Dataset (UCDP GED)  
3    UCDP G

In [146]:
#match the variable names with the descriptions
## you can change it to match the descriptions, so "if 'wdi' in df_name: header = 'world development index'"
## you can repeat it for as many columns you need 
header= ''
def give_me_shortdescription(df_new):
    global header
    if "ged_sb" in df_new: header = 'The best (most likely) estimate of the total number of battle-related deaths (BRDs) from state-based conflict as per the UCDP definition.'
    if "ged_ns" in df_new: header = "The best (most likely) estimate of the total number of fatalities from non-state conflict as per the UCDP definition."
    if "ged_os" in df_new: header = "The best (most likely) estimate of the total number of fatalities from one-sided conflict as per the UCDP definition."
    if "acled_sb" in df_new: header = "State based conflict events count from the event types battle and remote violence, where one actor is a military force and non of the involved actors are civilians."
    if "acled_sb_count" in df_new: header = "State based conflict events count from the event types battle and remote violence, where one actor is a military force and non of the involved actors are civilians."
    if "acled_os" in df_new: header = "Fatality count from the event types battle and remote violence, where one actor is civilians."
    if "wdi_sp_pop_totl" in df_new: header = "Population in total."
    if "wdi_ag_lnd_frst" in df_new: header = "Forest area (sq. km)."
    if "wdi_dt_oda_odat_pc_zs" in df_new: header = "Net official development assistance ODA received per capita in current US dollars."
    if "wdi_ms_mil_xpnd_gd_zs" in df_new: header = "Military expenditure as percentage of GDP."   
    if "wdi_ms_mil_xpnd_zs" in df_new: header = "Military expenditure as percentage of general government expenditure." 
    if "wdi_nv_agr_totl_kd" in df_new: header = "Value added from agriculture in constant 2015 U.S. dollars."
    if "wdi_nv_agr_totl_kn" in df_new: header = "Value added from agriculture in constant local currency." 
    if "wdi_ny_gdp_pcap_kd" in df_new: header = "GDP per capita in constant 2015 US dollars" 
    if "wdi_sp_dyn_le00_in" in df_new: header = "Life expectancy at birth, total years" 
    if "wdi_se_enr_prim_fm_zs" in df_new: header = "Gender parity index for gross enrollment ratio in primary education is the ratio of girls to boys enrolled at primary level in public and private schools." 
    if "wdi_se_enr_prsc_fm_zs" in df_new: header = "Gender parity index for gross enrollment ratio in primary and secondary education is the ratio of girls to boys enrolled at primary and secondary levels in public and private schools." 
    if "wdi_se_prm_nenr" in df_new: header = "School enrollment, primary percentage net" 
    if "wdi_sh_sta_maln_zs" in df_new: header = "Prevalence of underweight amongst children under 5 in percentage."
    if "wdi_sh_sta_stnt_zs" in df_new: header = "Prevalence of stunting amongst children under 5 in percentage."
    if "wdi_sl_tlf_totl_fe_zs" in df_new: header = "Female labor force as a percentage of the total."
    if "wdi_sm_pop_refg_or" in df_new: header = "Refugee population by country or territory of origin."
    if "wdi_sm_pop_netm" in df_new: header = "Net total of migrants during a five-year estimate."
    if "wdi_sm_pop_totl_zs" in df_new: header = "International migrant stock (number of people born in a country other than that in which they live) in percentage of population."
    if "wdi_sp_dyn_imrt_in" in df_new: header = "Infant mortality rate per 1,000 live births."
    if "wdi_sh_dyn_mort_fe" in df_new: header = "Female under-five mortality rate per 1,000 live birth."
    if "wdi_sp_pop_14_fe_zs" in df_new: header = "Female population between the ages 0 to 14 as a percentage of the total female population"
    if "wdi_sp_pop_1564_fe_zs" in df_new: header = "Female population between the ages 15 to 64 as a percentage of the total female population."
    if "wdi_sp_pop_65up_fe_zs" in df_new: header = "Female population 65 years of age or older as a percentage of the total female population."
    if "wdi_sp_pop_grow" in df_new: header = "Annual population growth in percentage."
    if "wdi_sp_urb_totl_in_zs" in df_new: header = "Percentage of population living in urban population areas."
    if "vdem_v2x_delibdem" in df_new: header = "Deliberative democracy index on an interval from low to high (0-1)."
    if "vdem_v2x_egaldem" in df_new: header = "Egalitarian democracy index on an interval from low to high (0-1)."
    if "vdem_v2x_libdem" in df_new: header = "Liberal democracy index on an interval from low to high (0-1)."
    if "vdem_v2x_partip" in df_new: header = "Participatory component index on an interval from low to high (0-1)."
    if "vdem_v2x_partipdem" in df_new: header = "Participatory democracy index on an interval from low to high (0-1)."
    if "vdem_v2x_accountability" in df_new: header = "Accountability index on a scale using a standard normal cumulative distribution function. It is thus scaled low to high (0-1)."
    if "vdem_v2x_civlib" in df_new: header = "Civil liberties index on an interval from low to high (0-1)."
    if "vdem_v2x_clphy" in df_new: header = "Physical violence index on an interval from low to high (0-1)."
    if "vdem_v2x_cspart" in df_new: header = "Civil society participation index on an interval from low to high (0-1)."
    if "vdem_v2x_divparctrl" in df_new: header = "Divided party control index on an interval from low to high."
    if "vdem_v2x_edcomp_thick" in df_new: header = "Electoral component index on an interval from low to high (0-1)."
    if "vdem_v2x_egal" in df_new: header = "Egalitarian component index on an interval from low to high (0-1)."
    if "vdem_v2x_execorr" in df_new: header = "Executive corruption index on an interval from low to high (0-1)."
    if "vdem_v2x_frassoc_thick" in df_new: header = "Freedom of association thick index on an interval from low to high (0-1)."
    if "vdem_v2x_gencs" in df_new: header = "Women civil society participation index on an interval from low to high (0-1)."
    if "vdem_v2x_gender" in df_new: header = "Women political empowerment index on an interval from low to high (0-1)."
    if "vdem_v2x_genpp" in df_new: header = "Women political participation index on an interval from low to high (0-1)."
    if "vdem_v2x_horacc" in df_new: header = "Horizontal accountability index on an interval from low to high (0-1)."
    if "vdem_v2x_neopat" in df_new: header = "Neopatrimonial Rule Index on an interval from low to high (0-1)."
    if "vdem_v2x_pubcorr" in df_new: header = "Public sector corruption index on an interval from low to high (0-1)."
    if "vdem_v2x_rule" in df_new: header = "Rule of law index on an interval from low to high (0-1)." 
    if "vdem_v2x_veracc" in df_new: header = "Vertical accountability index on an interval from low to high (0-1)."
    if "vdem_v2x_ex_military" in df_new: header = "Military dimension index on an interval from low to high (0-1)."
    if "vdem_v2x_ex_party" in df_new: header = "Ruling party dimension index on an interval from low to high (0-1)."
    if "vdem_v2x_freexp" in df_new: header = "Freedom of expression index on an interval from low to high (0-1)."
    if "vdem_v2xcl_acjst" in df_new: header = "Access to justice on an interval from low to high (0-1)."
    if "vdem_v2xcl_dmove" in df_new: header = "Freedom of domestic movement on an interval from low to high (0-1)."
    if "vdem_v2xcl_prpty" in df_new: header = "Property rights index on an interval from low to high (0-1)."
    if "vdem_v2xcl_rol" in df_new: header = "Equality before the law and individual liberty index on an interval from low to high (0-1)."
    if "vdem_v2xcl_slave" in df_new: header = "Freedom from forced labor index on an interval from low to high (0-1)."
    if "vdem_v2xdd_dd" in df_new: header = "Direct popular vote index on an interval from low to high (0-1)."
    if "vdem_v2xdl_delib" in df_new: header = "Deliberative component index on an interval from low to high (0-1)."
    if "vdem_v2xeg_eqdr" in df_new: header = "Equal distribution of resources index on an interval from low to high (0-1)."
    if "vdem_v2xeg_eqprotec" in df_new: header = "Equal protection index on an interval from low to high (0-1)."
    if "vdem_v2xel_frefair" in df_new: header = "Clean elections index on an interval from low to high (0-1)."
    if "vdem_v2xel_regelec" in df_new: header = "Regional government index on an interval from low to high (0-1)."
    if "vdem_v2xme_altinf" in df_new: header = "Regional government index on an interval from low to high (0-1)."
    if "vdem_v2xnp_client" in df_new: header = "Clientelism Index on an interval from low to high (0-1)."
    if "vdem_v2xnp_regcorr" in df_new: header = "Regime corruption Index on an interval from low to high (0-1)."
    if "vdem_v2xpe_exlecon" in df_new: header = "Exclusion by Socio-Economic Group Index on an interval from low to high (0-1)."
    if "vdem_v2xpe_exlpol" in df_new: header = "Exclusion by Political Group index on an interval from low to high (0-1)." 
    if "vdem_v2xpe_exlgeo" in df_new: header = "Exclusion by Urban-Rural Location index on an interval from low to high (0-1)." 
    if "vdem_v2xpe_exlgender" in df_new: header = "Exclusion by Gender index on an interval from low to high (0-1)."
    if "vdem_v2xpe_exlsocgr" in df_new: header = "Exclusion by Social Group index on an interval from low to high (0-1)."
    if "vdem_v2xps_party" in df_new: header = "Party institutionalization index on an interval from low to high (0-1)."
    if "vdem_v2xcs_ccsi" in df_new: header = "Core civil society index on an interval from low to high (0-1)." 
    if "vdem_v2xnp_pres" in df_new: header = "Presidentialism Index on an interval from low to high (0-1)." 
    if "vdem_v2xeg_eqaccess" in df_new: header = "Presidentialism Index on an interval from low to high (0-1)." 
    if "vdem_v2x_diagacc" in df_new: header = "Diagonal accountability index on an interval from low to high (0-1)."
    if "vdem_v2clrgunev" in df_new: header = "Does government respect for civil liberties vary across different areas of the country? 0: Yes, 1: Somewhat, 2: no." 
    if "agr_withdrawal_pct" in df_new: header = "Agricultural water withdrawal as percentage of total renewable water resources"
    if "dam_cap_pcap" in df_new: header = "Total dam storage capacity per capita [m3/inhab]"
    if "groundwater_export" in df_new: header = "Average annual quantity of groundwater leaving the country (total) 10^9 m3/year" 
    if "fresh_withdrawal_pct" in df_new: header = "Freshwater withdrawal as percentage of total renewable water resources"
    if "ind_efficiency" in df_new: header = "Industrial Water Use Efficiency in US dollars/m3"
    if "irr_agr_efficiency" in df_new: header = "Irrigated Agriculture Water Use Efficiency in US dollars/m3"
    if "services_efficiency" in df_new: header = "Services Water Use Efficiency in US dollars/m3"
    if "general_efficiency" in df_new: header = "Water Use Efficiency in US dollars/m3"
    if "water_stress" in df_new: header = "Freshwater withdrawal as a proportion of available freshwater resources, Water Stress"
    if "renewable_internal_pcap" in df_new: header = "Total internal renewable water resources per capita m3/inhab/yr"
    if "renewable_pcap" in df_new: header = "Total annual renewable water resources per capita m3/inhab/year"    
    return header

In [147]:
df_new['shortdescription'] = df_new['variable_name'].apply(give_me_shortdescription)
print(df_new)

                   variable_name        transform_description  \
0                 gleditsch_ward                                
1                         ged_sb                                
2                         ged_ns                                
3                         ged_os                                
4                       acled_sb                                
..                           ...                          ...   
121  renewable_internal_pcap_t48    temporal lag of 48 months   
122           renewable_pcap_t48    temporal lag of 48 months   
123       splag_1_decay_ged_sb_5  spatial lag, decay function   
124       splag_1_decay_ged_os_5  spatial lag, decay function   
125       splag_1_decay_ged_ns_5  spatial lag, decay function   

                               source_description  \
0                                                   
1    UCDP Georeferenced Events Dataset (UCDP GED)   
2    UCDP Georeferenced Events Dataset (UCDP GED)   
3    UC

In [173]:
df_new

,variable_name,transform_description,source_description,shortdescription
0,gleditsch_ward,,,
1,ged_sb,,UCDP Georeferenced Events Dataset (UCDP GED),The best (most likely) estimate of the total n...
2,ged_ns,,UCDP Georeferenced Events Dataset (UCDP GED),The best (most likely) estimate of the total n...
3,ged_os,,UCDP Georeferenced Events Dataset (UCDP GED),The best (most likely) estimate of the total n...
4,acled_sb,,Armed Conflict Location Event Dataset,State based conflict events count from the eve...
...,...,...,...,...
121,renewable_internal_pcap_t48,temporal lag of 48 months,FAO. [2022]. AQUASTAT Core Database,Total internal renewable water resources per c...
122,renewable_pcap_t48,temporal lag of 48 months,FAO. [2022]. AQUASTAT Core Database,Total annual renewable water resources per cap...
123,splag_1_decay_ged_sb_5,"spatial lag, decay function",UCDP Georeferenced Events Dataset (UCDP GED),The best (most likely) estimate of the total n...
124,splag_1_decay_ged_os_5,"spatial lag, decay function",UCDP Georeferenced Events Dataset (UCDP GED),The best (most likely) estimate of the total n...


In [174]:
json_file = df_new.to_json(orient="index")

In [175]:
json_file

'{"0":{"variable_name":"gleditsch_ward","transform_description":"","source_description":"","shortdescription":""},"1":{"variable_name":"ged_sb","transform_description":"","source_description":"UCDP Georeferenced Events Dataset (UCDP GED)","shortdescription":"The best (most likely) estimate of the total number of battle-related deaths (BRDs) from state-based conflict as per the UCDP definition."},"2":{"variable_name":"ged_ns","transform_description":"","source_description":"UCDP Georeferenced Events Dataset (UCDP GED)","shortdescription":"The best (most likely) estimate of the total number of fatalities from non-state conflict as per the UCDP definition."},"3":{"variable_name":"ged_os","transform_description":"","source_description":"UCDP Georeferenced Events Dataset (UCDP GED)","shortdescription":"The best (most likely) estimate of the total number of fatalities from one-sided conflict as per the UCDP definition."},"4":{"variable_name":"acled_sb","transform_description":"","source_desc

In [179]:
# Convert the JSON data to a string
json_str = json.dumps(json_file)

# Save the JSON string to a file
with open('/Users/sofia/VIEWS3/Test/codebook_generator/cm_features_competition_desc.json', 'w') as f:
    f.write(json_str)

In [180]:
# Save the DataFrame to a CSV file
df_new.to_csv('my_table.csv', index=False)

# Read the CSV file back in as a DataFrame
df = pd.read_csv('my_table.csv')

# Print the DataFrame
print(df)

                   variable_name        transform_description  \
0                 gleditsch_ward                          NaN   
1                         ged_sb                          NaN   
2                         ged_ns                          NaN   
3                         ged_os                          NaN   
4                       acled_sb                          NaN   
..                           ...                          ...   
121  renewable_internal_pcap_t48    temporal lag of 48 months   
122           renewable_pcap_t48    temporal lag of 48 months   
123       splag_1_decay_ged_sb_5  spatial lag, decay function   
124       splag_1_decay_ged_os_5  spatial lag, decay function   
125       splag_1_decay_ged_ns_5  spatial lag, decay function   

                               source_description  \
0                                             NaN   
1    UCDP Georeferenced Events Dataset (UCDP GED)   
2    UCDP Georeferenced Events Dataset (UCDP GED)   
3    UC

In [181]:
df

,variable_name,transform_description,source_description,shortdescription
0,gleditsch_ward,NaN,NaN,NaN
1,ged_sb,NaN,UCDP Georeferenced Events Dataset (UCDP GED),The best (most likely) estimate of the total n...
2,ged_ns,NaN,UCDP Georeferenced Events Dataset (UCDP GED),The best (most likely) estimate of the total n...
3,ged_os,NaN,UCDP Georeferenced Events Dataset (UCDP GED),The best (most likely) estimate of the total n...
4,acled_sb,NaN,Armed Conflict Location Event Dataset,State based conflict events count from the eve...
...,...,...,...,...
121,renewable_internal_pcap_t48,temporal lag of 48 months,FAO. [2022]. AQUASTAT Core Database,Total internal renewable water resources per c...
122,renewable_pcap_t48,temporal lag of 48 months,FAO. [2022]. AQUASTAT Core Database,Total annual renewable water resources per cap...
123,splag_1_decay_ged_sb_5,"spatial lag, decay function",UCDP Georeferenced Events Dataset (UCDP GED),The best (most likely) estimate of the total n...
124,splag_1_decay_ged_os_5,"spatial lag, decay function",UCDP Georeferenced Events Dataset (UCDP GED),The best (most likely) estimate of the total n...


In [182]:
# Convert the DataFrame to a Markdown-formatted string
md_table = df_new.to_markdown()



In [183]:
md_table

'|     | variable_name                 | transform_description       | source_description                           | shortdescription                                                                                                                                                                        |\n|----:|:------------------------------|:----------------------------|:---------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|\n|   0 | gleditsch_ward                |                             |                                              |                                                                                                                                                                                         |\n|   1 | ged_sb                        |                             | UCDP Georeference

In [184]:
# Write the Markdown string to a file
with open("my_table.md", "w") as f:
    f.write(md_table)

In [172]:
md_table

'|     | variable_name                 | transform_description       | source_description                           | shortdescription                                                                                                                                                                        |\n|----:|:------------------------------|:----------------------------|:---------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|\n|   0 | gleditsch_ward                | nan                         | nan                                          | nan                                                                                                                                                                                     |\n|   1 | ged_sb                        | nan                         | UCDP Georeference

In [158]:
print(df_new.to_markdown())

|     | variable_name                 | transform_description       | source_description                           | shortdescription                                                                                                                                                                        |
|----:|:------------------------------|:----------------------------|:---------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|   0 | gleditsch_ward                |                             |                                              |                                                                                                                                                                                         |
|   1 | ged_sb                        |                             | UCDP Georeferenced Ev

|     | variable_name                 | transform_description       | source_description                           | shortdescription                                                                                                                                                                        |
|----:|:------------------------------|:----------------------------|:---------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|   0 | gleditsch_ward                |                             |                                              |                                                                                                                                                                                         |
|   1 | ged_sb                        |                             | UCDP Georeferenced Events Dataset (UCDP GED) | The best (most likely) estimate of the total number of battle-related deaths (BRDs) from state-based conflict as per the UCDP definition.                                               |
|   2 | ged_ns                        |                             | UCDP Georeferenced Events Dataset (UCDP GED) | The best (most likely) estimate of the total number of fatalities from non-state conflict as per the UCDP definition.                                                                   |
|   3 | ged_os                        |                             | UCDP Georeferenced Events Dataset (UCDP GED) | The best (most likely) estimate of the total number of fatalities from one-sided conflict as per the UCDP definition.                                                                   |
|   4 | acled_sb                      |                             | Armed Conflict Location Event Dataset        | State based conflict events count from the event types battle and remote violence, where one actor is a military force and non of the involved actors are civilians.                    |
|   5 | acled_sb_count                |                             | Armed Conflict Location Event Dataset        | State based conflict events count from the event types battle and remote violence, where one actor is a military force and non of the involved actors are civilians.                    |
|   6 | acled_os                      |                             | Armed Conflict Location Event Dataset        | Fatality count from the event types battle and remote violence, where one actor is civilians.                                                                                           |
|   7 | ged_sb_tsum_24                | 24-month moving sum         | UCDP Georeferenced Events Dataset (UCDP GED) | The best (most likely) estimate of the total number of battle-related deaths (BRDs) from state-based conflict as per the UCDP definition.                                               |
|   8 | wdi_sp_pop_totl               |                             | World Development Index                      | Population in total.                                                                                                                                                                    |
|   9 | ged_sb_tlag_1                 | temporal lag of X months    | UCDP Georeferenced Events Dataset (UCDP GED) | The best (most likely) estimate of the total number of battle-related deaths (BRDs) from state-based conflict as per the UCDP definition.                                               |
|  10 | ged_sb_tlag_2                 | temporal lag of X months    | UCDP Georeferenced Events Dataset (UCDP GED) | The best (most likely) estimate of the total number of battle-related deaths (BRDs) from state-based conflict as per the UCDP definition.                                               |
|  11 | ged_sb_tlag_3                 | temporal lag of X months    | UCDP Georeferenced Events Dataset (UCDP GED) | The best (most likely) estimate of the total number of battle-related deaths (BRDs) from state-based conflict as per the UCDP definition.                                               |
|  12 | ged_sb_tlag_4                 | temporal lag of X months    | UCDP Georeferenced Events Dataset (UCDP GED) | The best (most likely) estimate of the total number of battle-related deaths (BRDs) from state-based conflict as per the UCDP definition.                                               |
|  13 | ged_sb_tlag_5                 | temporal lag of X months    | UCDP Georeferenced Events Dataset (UCDP GED) | The best (most likely) estimate of the total number of battle-related deaths (BRDs) from state-based conflict as per the UCDP definition.                                               |
|  14 | ged_sb_tlag_6                 | temporal lag of X months    | UCDP Georeferenced Events Dataset (UCDP GED) | The best (most likely) estimate of the total number of battle-related deaths (BRDs) from state-based conflict as per the UCDP definition.                                               |
|  15 | ged_os_tlag_1                 | temporal lag of X months    | UCDP Georeferenced Events Dataset (UCDP GED) | The best (most likely) estimate of the total number of fatalities from one-sided conflict as per the UCDP definition.                                                                   |
|  16 | wdi_ag_lnd_frst_k2            |                             | World Development Index                      | Forest area (sq. km).                                                                                                                                                                   |
|  17 | wdi_dt_oda_odat_pc_zs         |                             | World Development Index                      | Net official development assistance ODA received per capita in current US dollars.                                                                                                      |
|  18 | wdi_ms_mil_xpnd_gd_zs         |                             | World Development Index                      | Military expenditure as percentage of GDP.                                                                                                                                              |
|  19 | wdi_ms_mil_xpnd_zs            |                             | World Development Index                      | Military expenditure as percentage of general government expenditure.                                                                                                                   |
|  20 | wdi_nv_agr_totl_kd            |                             | World Development Index                      | Value added from agriculture in constant 2015 U.S. dollars.                                                                                                                             |
|  21 | wdi_nv_agr_totl_kn            |                             | World Development Index                      | Value added from agriculture in constant local currency.                                                                                                                                |
|  22 | wdi_ny_gdp_pcap_kd            |                             | World Development Index                      | GDP per capita in constant 2015 US dollars                                                                                                                                              |
|  23 | wdi_sp_dyn_le00_in            |                             | World Development Index                      | Life expectancy at birth, total years                                                                                                                                                   |
|  24 | wdi_se_enr_prim_fm_zs         |                             | World Development Index                      | Gender parity index for gross enrollment ratio in primary education is the ratio of girls to boys enrolled at primary level in public and private schools.                              |
|  25 | wdi_se_enr_prsc_fm_zs         |                             | World Development Index                      | Gender parity index for gross enrollment ratio in primary and secondary education is the ratio of girls to boys enrolled at primary and secondary levels in public and private schools. |
|  26 | wdi_se_prm_nenr               |                             | World Development Index                      | School enrollment, primary percentage net                                                                                                                                               |
|  27 | wdi_sh_sta_maln_zs            |                             | World Development Index                      | Prevalence of underweight amongst children under 5 in percentage.                                                                                                                       |
|  28 | wdi_sh_sta_stnt_zs            |                             | World Development Index                      | Prevalence of stunting amongst children under 5 in percentage.                                                                                                                          |
|  29 | wdi_sl_tlf_totl_fe_zs         |                             | World Development Index                      | Female labor force as a percentage of the total.                                                                                                                                        |
|  30 | wdi_sm_pop_refg_or            |                             | World Development Index                      | Refugee population by country or territory of origin.                                                                                                                                   |
|  31 | wdi_sm_pop_netm               |                             | World Development Index                      | Net total of migrants during a five-year estimate.                                                                                                                                      |
|  32 | wdi_sm_pop_totl_zs            |                             | World Development Index                      | International migrant stock (number of people born in a country other than that in which they live) in percentage of population.                                                        |
|  33 | wdi_sp_dyn_imrt_in            |                             | World Development Index                      | Infant mortality rate per 1,000 live births.                                                                                                                                            |
|  34 | wdi_sh_dyn_mort_fe            |                             | World Development Index                      | Female under-five mortality rate per 1,000 live birth.                                                                                                                                  |
|  35 | wdi_sp_pop_14_fe_zs           |                             | World Development Index                      | Female population between the ages 0 to 14 as a percentage of the total female population                                                                                               |
|  36 | wdi_sp_pop_1564_fe_zs         |                             | World Development Index                      | Female population between the ages 15 to 64 as a percentage of the total female population.                                                                                             |
|  37 | wdi_sp_pop_65up_fe_zs         |                             | World Development Index                      | Female population 65 years of age or older as a percentage of the total female population.                                                                                              |
|  38 | wdi_sp_pop_grow               |                             | World Development Index                      | Annual population growth in percentage.                                                                                                                                                 |
|  39 | wdi_sp_urb_totl_in_zs         |                             | World Development Index                      | Percentage of population living in urban population areas.                                                                                                                              |
|  40 | splag_wdi_sl_tlf_totl_fe_zs   | spatial lag                 | World Development Index                      | Female labor force as a percentage of the total.                                                                                                                                        |
|  41 | splag_wdi_sm_pop_refg_or      | spatial lag                 | World Development Index                      | Refugee population by country or territory of origin.                                                                                                                                   |
|  42 | splag_wdi_sm_pop_netm         | spatial lag                 | World Development Index                      | Net total of migrants during a five-year estimate.                                                                                                                                      |
|  43 | splag_wdi_ag_lnd_frst_k2      | spatial lag                 | World Development Index                      | Forest area (sq. km).                                                                                                                                                                   |
|  44 | vdem_v2x_delibdem             |                             | Varieties of Democracy (vdem)                | Deliberative democracy index on an interval from low to high (0-1).                                                                                                                     |
|  45 | vdem_v2x_egaldem              |                             | Varieties of Democracy (vdem)                | Egalitarian component index on an interval from low to high (0-1).                                                                                                                      |
|  46 | vdem_v2x_libdem               |                             | Varieties of Democracy (vdem)                | Liberal democracy index on an interval from low to high (0-1).                                                                                                                          |
|  47 | vdem_v2x_libdem_48            |                             | Varieties of Democracy (vdem)                | Liberal democracy index on an interval from low to high (0-1).                                                                                                                          |
|  48 | vdem_v2x_partip               |                             | Varieties of Democracy (vdem)                | Participatory component index on an interval from low to high (0-1).                                                                                                                    |
|  49 | vdem_v2x_partipdem            |                             | Varieties of Democracy (vdem)                | Participatory democracy index on an interval from low to high (0-1).                                                                                                                    |
|  50 | vdem_v2x_accountability       |                             | Varieties of Democracy (vdem)                | Accountability index on a scale using a standard normal cumulative distribution function. It is thus scaled low to high (0-1).                                                          |
|  51 | vdem_v2x_civlib               |                             | Varieties of Democracy (vdem)                | Civil liberties index on an interval from low to high (0-1).                                                                                                                            |
|  52 | vdem_v2x_clphy                |                             | Varieties of Democracy (vdem)                | Physical violence index on an interval from low to high (0-1).                                                                                                                          |
|  53 | vdem_v2x_cspart               |                             | Varieties of Democracy (vdem)                | Civil society participation index on an interval from low to high (0-1).                                                                                                                |
|  54 | vdem_v2x_divparctrl           |                             | Varieties of Democracy (vdem)                | Divided party control index on an interval from low to high.                                                                                                                            |
|  55 | vdem_v2x_edcomp_thick         |                             | Varieties of Democracy (vdem)                | Electoral component index on an interval from low to high (0-1).                                                                                                                        |
|  56 | vdem_v2x_egal                 |                             | Varieties of Democracy (vdem)                | Egalitarian component index on an interval from low to high (0-1).                                                                                                                      |
|  57 | vdem_v2x_execorr              |                             | Varieties of Democracy (vdem)                | Executive corruption index on an interval from low to high (0-1).                                                                                                                       |
|  58 | vdem_v2x_frassoc_thick        |                             | Varieties of Democracy (vdem)                | Freedom of association thick index on an interval from low to high (0-1).                                                                                                               |
|  59 | vdem_v2x_gencs                |                             | Varieties of Democracy (vdem)                | Women civil society participation index on an interval from low to high (0-1).                                                                                                          |
|  60 | vdem_v2x_gender               |                             | Varieties of Democracy (vdem)                | Women political empowerment index on an interval from low to high (0-1).                                                                                                                |
|  61 | vdem_v2x_genpp                |                             | Varieties of Democracy (vdem)                | Women political participation index on an interval from low to high (0-1).                                                                                                              |
|  62 | vdem_v2x_horacc               |                             | Varieties of Democracy (vdem)                | Horizontal accountability index on an interval from low to high (0-1).                                                                                                                  |
|  63 | vdem_v2x_neopat               |                             | Varieties of Democracy (vdem)                | Neopatrimonial Rule Index on an interval from low to high (0-1).                                                                                                                        |
|  64 | vdem_v2x_pubcorr              |                             | Varieties of Democracy (vdem)                | Public sector corruption index on an interval from low to high (0-1).                                                                                                                   |
|  65 | vdem_v2x_rule                 |                             | Varieties of Democracy (vdem)                | Rule of law index on an interval from low to high (0-1).                                                                                                                                |
|  66 | vdem_v2x_veracc               |                             | Varieties of Democracy (vdem)                | Vertical accountability index on an interval from low to high (0-1).                                                                                                                    |
|  67 | vdem_v2x_ex_military          |                             | Varieties of Democracy (vdem)                | Military dimension index on an interval from low to high (0-1).                                                                                                                         |
|  68 | vdem_v2x_ex_party             |                             | Varieties of Democracy (vdem)                | Ruling party dimension index on an interval from low to high (0-1).                                                                                                                     |
|  69 | vdem_v2x_freexp               |                             | Varieties of Democracy (vdem)                | Freedom of expression index on an interval from low to high (0-1).                                                                                                                      |
|  70 | vdem_v2xcl_acjst              |                             | Varieties of Democracy (vdem)                | Access to justice on an interval from low to high (0-1).                                                                                                                                |
|  71 | vdem_v2xcl_dmove              |                             | Varieties of Democracy (vdem)                | Freedom of domestic movement on an interval from low to high (0-1).                                                                                                                     |
|  72 | vdem_v2xcl_prpty              |                             | Varieties of Democracy (vdem)                | Property rights index on an interval from low to high (0-1).                                                                                                                            |
|  73 | vdem_v2xcl_rol                |                             | Varieties of Democracy (vdem)                | Equality before the law and individual liberty index on an interval from low to high (0-1).                                                                                             |
|  74 | vdem_v2xcl_slave              |                             | Varieties of Democracy (vdem)                | Freedom from forced labor index on an interval from low to high (0-1).                                                                                                                  |
|  75 | vdem_v2xdd_dd                 |                             | Varieties of Democracy (vdem)                | Direct popular vote index on an interval from low to high (0-1).                                                                                                                        |
|  76 | vdem_v2xdl_delib              |                             | Varieties of Democracy (vdem)                | Deliberative component index on an interval from low to high (0-1).                                                                                                                     |
|  77 | vdem_v2xeg_eqdr               |                             | Varieties of Democracy (vdem)                | Equal distribution of resources index on an interval from low to high (0-1).                                                                                                            |
|  78 | vdem_v2xeg_eqprotec           |                             | Varieties of Democracy (vdem)                | Equal protection index on an interval from low to high (0-1).                                                                                                                           |
|  79 | vdem_v2xel_frefair            |                             | Varieties of Democracy (vdem)                | Clean elections index on an interval from low to high (0-1).                                                                                                                            |
|  80 | vdem_v2xel_regelec            |                             | Varieties of Democracy (vdem)                | Regional government index on an interval from low to high (0-1).                                                                                                                        |
|  81 | vdem_v2xme_altinf             |                             | Varieties of Democracy (vdem)                | Regional government index on an interval from low to high (0-1).                                                                                                                        |
|  82 | vdem_v2xnp_client             |                             | Varieties of Democracy (vdem)                | Clientelism Index on an interval from low to high (0-1).                                                                                                                                |
|  83 | vdem_v2xnp_regcorr            |                             | Varieties of Democracy (vdem)                | Regime corruption Index on an interval from low to high (0-1).                                                                                                                          |
|  84 | vdem_v2xpe_exlecon            |                             | Varieties of Democracy (vdem)                | Exclusion by Socio-Economic Group Index on an interval from low to high (0-1).                                                                                                          |
|  85 | vdem_v2xpe_exlpol             |                             | Varieties of Democracy (vdem)                | Exclusion by Political Group index on an interval from low to high (0-1).                                                                                                               |
|  86 | vdem_v2xpe_exlgeo             |                             | Varieties of Democracy (vdem)                | Exclusion by Urban-Rural Location index on an interval from low to high (0-1).                                                                                                          |
|  87 | vdem_v2xpe_exlgender          |                             | Varieties of Democracy (vdem)                | Exclusion by Gender index on an interval from low to high (0-1).                                                                                                                        |
|  88 | vdem_v2xpe_exlsocgr           |                             | Varieties of Democracy (vdem)                | Exclusion by Social Group index on an interval from low to high (0-1).                                                                                                                  |
|  89 | vdem_v2xps_party              |                             | Varieties of Democracy (vdem)                | Party institutionalization index on an interval from low to high (0-1).                                                                                                                 |
|  90 | vdem_v2xcs_ccsi               |                             | Varieties of Democracy (vdem)                | Core civil society index on an interval from low to high (0-1).                                                                                                                         |
|  91 | vdem_v2xnp_pres               |                             | Varieties of Democracy (vdem)                | Presidentialism Index on an interval from low to high (0-1).                                                                                                                            |
|  92 | vdem_v2xeg_eqaccess           |                             | Varieties of Democracy (vdem)                | Presidentialism Index on an interval from low to high (0-1).                                                                                                                            |
|  93 | vdem_v2x_diagacc              |                             | Varieties of Democracy (vdem)                | Diagonal accountability index on an interval from low to high (0-1).                                                                                                                    |
|  94 | vdem_v2clrgunev               |                             | Varieties of Democracy (vdem)                | Does government respect for civil liberties vary across different areas of the country? 0: Yes, 1: Somewhat, 2: no.                                                                     |
|  95 | splag_vdem_v2x_libdem         | spatial lag                 | Varieties of Democracy (vdem)                | Liberal democracy index on an interval from low to high (0-1).                                                                                                                          |
|  96 | splag_vdem_v2xcl_dmove        | spatial lag                 | Varieties of Democracy (vdem)                | Freedom of domestic movement on an interval from low to high (0-1).                                                                                                                     |
|  97 | splag_vdem_v2x_accountability | spatial lag                 | Varieties of Democracy (vdem)                | Accountability index on a scale using a standard normal cumulative distribution function. It is thus scaled low to high (0-1).                                                          |
|  98 | splag_vdem_v2xpe_exlsocgr     | spatial lag                 | Varieties of Democracy (vdem)                | Exclusion by Social Group index on an interval from low to high (0-1).                                                                                                                  |
|  99 | splag_vdem_v2xcl_rol          | spatial lag                 | Varieties of Democracy (vdem)                | Equality before the law and individual liberty index on an interval from low to high (0-1).                                                                                             |
| 100 | _wdi_sm_pop_netm              |                             | World Development Index                      | Net total of migrants during a five-year estimate.                                                                                                                                      |
| 101 | _wdi_sp_dyn_imrt_in           |                             | World Development Index                      | Infant mortality rate per 1,000 live births.                                                                                                                                            |
| 102 | decay_ged_sb_5                | decay function              | UCDP Georeferenced Events Dataset (UCDP GED) | The best (most likely) estimate of the total number of battle-related deaths (BRDs) from state-based conflict as per the UCDP definition.                                               |
| 103 | decay_ged_os_5                | decay function              | UCDP Georeferenced Events Dataset (UCDP GED) | The best (most likely) estimate of the total number of fatalities from one-sided conflict as per the UCDP definition.                                                                   |
| 104 | decay_ged_sb_100              | decay function              | UCDP Georeferenced Events Dataset (UCDP GED) | The best (most likely) estimate of the total number of battle-related deaths (BRDs) from state-based conflict as per the UCDP definition.                                               |
| 105 | decay_ged_sb_500              | decay function              | UCDP Georeferenced Events Dataset (UCDP GED) | The best (most likely) estimate of the total number of battle-related deaths (BRDs) from state-based conflict as per the UCDP definition.                                               |
| 106 | decay_ged_os_100              | decay function              | UCDP Georeferenced Events Dataset (UCDP GED) | The best (most likely) estimate of the total number of fatalities from one-sided conflict as per the UCDP definition.                                                                   |
| 107 | decay_ged_ns_5                | decay function              | UCDP Georeferenced Events Dataset (UCDP GED) | The best (most likely) estimate of the total number of fatalities from non-state conflict as per the UCDP definition.                                                                   |
| 108 | decay_ged_ns_100              | decay function              | UCDP Georeferenced Events Dataset (UCDP GED) | The best (most likely) estimate of the total number of fatalities from non-state conflict as per the UCDP definition.                                                                   |
| 109 | decay_acled_sb_5              | decay function              | Armed Conflict Location Event Dataset        | State based conflict events count from the event types battle and remote violence, where one actor is a military force and non of the involved actors are civilians.                    |
| 110 | decay_acled_os_5              | decay function              | Armed Conflict Location Event Dataset        | Fatality count from the event types battle and remote violence, where one actor is civilians.                                                                                           |
| 111 | decay_acled_ns_5              | decay function              | Armed Conflict Location Event Dataset        | Fatality count from the event types battle and remote violence, where one actor is civilians.                                                                                           |
| 112 | agr_withdrawal_pct_t48        | temporal lag of 48 months   | AQUASTAT Glossary, FAO, 2019                 | Agricultural water withdrawal as percentage of total renewable water resources                                                                                                          |
| 113 | dam_cap_pcap_t48              | temporal lag of 48 months   | AQUASTAT Glossary, FAO, 2019                 | Total dam storage capacity per capita [m3/inhab]                                                                                                                                        |
| 114 | groundwater_export_t48        | temporal lag of 48 months   | AQUASTAT Glossary, FAO, 2019                 | Average annual quantity of groundwater leaving the country (total) 10^9 m3/year                                                                                                         |
| 115 | fresh_withdrawal_pct_t48      | temporal lag of 48 months   | AQUASTAT Glossary, FAO, 2019                 | Freshwater withdrawal as percentage of total renewable water resources                                                                                                                  |
| 116 | ind_efficiency_t48            | temporal lag of 48 months   | FAO. [2022]. AQUASTAT Core Database          | Industrial Water Use Efficiency in US dollars/m3                                                                                                                                        |
| 117 | irr_agr_efficiency_t48        | temporal lag of 48 months   | FAO. [2022]. AQUASTAT Core Database          | Irrigated Agriculture Water Use Efficiency in US dollars/m3                                                                                                                             |
| 118 | services_efficiency_t48       | temporal lag of 48 months   | FAO. [2022]. AQUASTAT Core Database          | Services Water Use Efficiency in US dollars/m3                                                                                                                                          |
| 119 | general_efficiency_t48        | temporal lag of 48 months   | FAO. [2022]. AQUASTAT Core Database          | Water Use Efficiency in US dollars/m3                                                                                                                                                   |
| 120 | water_stress_t48              | temporal lag of 48 months   | FAO. [2022]. AQUASTAT Core Database          | Freshwater withdrawal as a proportion of available freshwater resources, Water Stress                                                                                                   |
| 121 | renewable_internal_pcap_t48   | temporal lag of 48 months   | FAO. [2022]. AQUASTAT Core Database          | Total internal renewable water resources per capita m3/inhab/yr                                                                                                                         |
| 122 | renewable_pcap_t48            | temporal lag of 48 months   | FAO. [2022]. AQUASTAT Core Database          | Total annual renewable water resources per capita m3/inhab/year                                                                                                                         |
| 123 | splag_1_decay_ged_sb_5        | spatial lag, decay function | UCDP Georeferenced Events Dataset (UCDP GED) | The best (most likely) estimate of the total number of battle-related deaths (BRDs) from state-based conflict as per the UCDP definition.                                               |
| 124 | splag_1_decay_ged_os_5        | spatial lag, decay function | UCDP Georeferenced Events Dataset (UCDP GED) | The best (most likely) estimate of the total number of fatalities from one-sided conflict as per the UCDP definition.                                                                   |
| 125 | splag_1_decay_ged_ns_5        | spatial lag, decay function | UCDP Georeferenced Events Dataset (UCDP GED) | The best (most likely) estimate of the total number of fatalities from non-state conflict as per the UCDP definition.                                                                   |